In [ ]:
%reload_ext autoreload
%autoreload 2

from flowquery import Runner

Create virtual node test

In [ ]:
query: str = """
    create virtual (:Fact) as {
        unwind range(0,10) as i
        load json from "https://catfact.ninja/fact" as item
        return i as id, item.fact
    }
"""

runner: Runner = Runner(query)
await runner.run()

Query virtual graph node

In [ ]:
runner: Runner = Runner("match (n:Fact) return n")
await runner.run()
for record in runner.results:
    print(record)